#### Nom : Daniel Joaquim
#### Prenóm : Paulino
#### Filiere : IDDL

# Subjet :  Question Answering 


## Étape 1 - Installation des bibliothèques requises

In [1]:
!pip install transformers 
!pip install torch 
!pip install simpletransformers
!pip install tensorflow
!pip install html2text --quiet

## Étape 2 : Importation des packages requis

In [2]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch
import numpy as np
from IPython.display import display
import html2text 
from IPython.html import widgets


/opt/anaconda3/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "




## Étape 3 : Charger le modèle Bert pré-entraîné

In [3]:
#,force_download=True

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad',force_download=True)

tokenizer_for_bert = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

## Étape 4 : Définir la fonction pour la réponse aux questions

In [10]:
def bert_question_answer(question, contexte, max_len=512):
    
    #"""
    #question: What is the name of YouTube Channel
    #context: Watch complete playlist of Natural Language Processing. Don't forget to like, share and subscribe my channel IG Tech Team
    #"""

    #Tokenize question d'entrée et text
    #Ajouter des Tokens spéciaux - [CLS] et [SEP]
        
    input_ids = tokenizer_for_bert.encode (question, contexte,  max_length= max_len, truncation=True)  
  
    #Obtenir le nombre de tokens dans la 1ère phrase (question) et la 2ème phrase (text qui contient la réponse)
    
    sep_index = input_ids.index(102) 
    len_question = sep_index + 1   
    len_passage = len(input_ids)- len_question  
    
    #Besoin de séparer question et text
     #Les identifiants de segment seront 0 pour la question et 1 pour le text
        
    segment_ids =  [0]*len_question + [1]*(len_passage)  

    #Conversion des identifiants de jetons en jetons
    
    tokens = tokenizer_for_bert.convert_ids_to_tokens(input_ids) 
   
    #Obtenir les scores de début et de fin pour la réponse
    #Conversion des tableaux d'entrée en tenseurs de torche avant de passer au modèle
        
    start_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]) )[0]
    end_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]) )[1]
    
    #Conversion des tenseurs de scores en tableaux numpy
    
    start_token_scores = start_token_scores.detach().numpy().flatten()
    end_token_scores = end_token_scores.detach().numpy().flatten()
        
   #Obtenir un index de réponse de début et de fin basé sur les scores les plus élevés

    answer_start_index = np.argmax(start_token_scores)
    answer_end_index = np.argmax(end_token_scores)

    #Obtenir des scores pour le jeton de début et de fin de la réponse
    
    start_token_score = np.round(start_token_scores[answer_start_index], 2)
    end_token_score = np.round(end_token_scores[answer_end_index], 2)

    #Combiner les sous-mots commençant par ## et obtenir des mots complets en sortie.
     #C'est parce que le tokenizer casse des mots qui ne sont pas dans son vocabulaire.
        
    answer = tokens[answer_start_index] 
    for i in range(answer_start_index + 1, answer_end_index + 1):
        if tokens[i][0:2] == '##':  
            answer += tokens[i][2:] 
        else:
            answer += ' ' + tokens[i]  

   # Si la réponse n'a pas été trouvée dans le passage

    if ( answer_start_index == 0) or (start_token_score < 0 ) or  (answer == '[SEP]') or ( answer_end_index <  answer_start_index):
        answer = "Sorry!, I could not find an answer in the contexte."
    
    return (answer_start_index, answer_end_index, start_token_score, end_token_score,  answer)

# Tester la Fonction

bert_question_answer("What is the name of YouTube Channel", "Watch complete playlist of Natural Language Processing. Don't forget to like, share and subscribe my channel IG Tech Team ")

(31, 34, 1.59, 1.43, 'ig tech team')



## Utilisons un autre passage et vérifions la sortie

   ### Question answering en Anglais 
   
   

In [5]:
# Let me define one passage
contexte = """Hello, I am Daniel. My friend name is Ajay. He is the son of Kristen. I spend most of the time with Ajay. 
He always call me by my nick name. Ajay call me programmer. Except Ajay, my other friend call me by my original name. 
Bijay is also my friend. """

print (f'Length of the context: {len(contexte.split())} words')

question1 ="What is my name?" 
print ('\nQuestion 1:\n', question1)
_, _ , _ , _, ans  = bert_question_answer( question1, contexte)
print('\nAnswer from BERT: ', ans ,  '\n')


question2 ="Who is the father of Ajay?"
print ('\nQuestion 2:\n', question2)
_, _ , _ , _, ans  = bert_question_answer( question2, contexte)
print('\nAnswer from BERT: ', ans ,  '\n')

question3 ="With whom Ishwar spend most of the time?" 
print ('\nQuestion 3:\n', question3)
_, _ , _ , _, ans  = bert_question_answer( question3, contexte)
print('\nAnswer from BERT: ', ans ,  '\n')

Length of the context: 51 words

Question 1:
 What is my name?

Answer from BERT:  daniel 


Question 2:
 Who is the father of Ajay?

Answer from BERT:  kristen 


Question 3:
 With whom Ishwar spend most of the time?

Answer from BERT:  ajay 



   ## Question answering en Français 
   
   


In [6]:
# Let me define another passage
contexte= """ La NLP est un sous-domaine de l'informatique et de l'intelligence artificielle qui s'intéresse aux interactions entre
ordinateurs et langages humains (naturels). Il est utilisé pour appliquer des algorithmes d'apprentissage automatique au texte et à 
la parole. Pour Par exemple, nous pouvons utiliser la NLP pour créer des systèmes tels que la reconnaissance vocale, la synthèse de 
documents, la traduction automatique, le spam détection, reconnaissance d'entité nommée, réponse aux questions, saisie semi-automatique, 
saisie prédictive, etc. De nos jours, la plupart des nous avons des smartphones dotés de la reconnaissance vocale. Ces smartphones utilisent
la NLP pour comprendre ce qui se dit. Aussi, beaucoup les gens utilisent des ordinateurs portables dont le système d'exploitation a une 
reconnaissance vocale intégrée. NLTK (Natural Language Toolkit) est un plate-forme leader pour la création de programmes Python pour travailler 
avec des données de langage humain. Il fournit des interfaces faciles à utiliser à de nombreux corpus et ressources lexicales. En outre, 
il contient une suite de bibliothèques de traitement de texte pour la classification, tokenisation, stemming, tagging, parsing et raisonnement
sémantique. Mieux encore, NLTK est un logiciel libre et open source, projet communautaire. Nous utiliserons cette boîte à outils pour montrer 
quelques bases du domaine du traitement du langage naturel. Pour
les exemples ci-dessous, je suppose que nous avons importé la boîte à outils NLTK. Nous pouvons faire ceci comme ceci : import nltk.
La tokenisation de la phrase (également appelée segmentation de la phrase) est le problème de diviser une chaîne de langage écrit en
ses phrases composantes. L'idée ici semble très simple. Tokenisation de mots (également appelée segmentation de mots)
est le problème de diviser une chaîne de langage écrit en ses mots composants. En anglais et dans de nombreuses autres langues
en utilisant une certaine forme d'alphabet latin, l'espace est une bonne approximation d'un séparateur de mots. Cependant, nous pouvons encore avoir des problèmes
nous ne divisons que par espace pour obtenir les résultats souhaités. Certains noms composés anglais sont écrits de manière variable et parfois
ils contiennent un espace. Dans la plupart des cas, nous utilisons une bibliothèque pour obtenir les résultats souhaités, donc encore une fois ne vous inquiétez pas trop
pour les détails."""

print (f'Longueur du contexte: {len(contexte.split())} mots')


question ="Quelle est la forme complète de NLTK ?"
print ('\nQuestion 1:\n', question)
_, _ , _ , _, ans  = bert_question_answer( question, contexte)
print('\nAnswer from BERT: ', ans ,  '\n')

question ="Qu'est-ce que la NLP ?"
print ('\nQuestion 2:\n', question)
_, _ , _ , _, ans  = bert_question_answer( question, contexte)
print('\nAnswer from BERT: ', ans ,  '\n')

question ="Qu'est-ce que l'apprentissage supervisé ?"
print ('\nQuestion 3:\n', question)
_, _ , _ , _, ans  = bert_question_answer( question, contexte)
print('\nAnswer from BERT: ', ans ,  '\n')


Longueur du contexte: 363 mots

Question 1:
 Quelle est la forme complète de NLTK ?

Answer from BERT:  un plate - forme leader pour la creation de programmes python pour travailler avec des donnees de langage humain 


Question 2:
 Qu'est-ce que la NLP ?

Answer from BERT:  un sous - domaine de l ' informatique et de l ' intelligence artificielle qui s ' interesse aux interactions entre ordinateurs et langages humains ( naturels ) 


Question 3:
 Qu'est-ce que l'apprentissage supervisé ?

Answer from BERT:  il est utilise pour appliquer des algorithmes d ' apprentissage automatique au texte et a la parole 




## Question answering en DRIJA¶


In [11]:
# Let me define another passage
contexte = """
          althaqafat almaghribiat hi mazij min alearab walbarbar 
          al'asliiyn wa'afriqia janub alsahra'i. allughat alrasmiat 
          lilmaghrib hi alearabiat wal'amazighiatu."""

print (f'tul almamari: {len(contexte.split())} kalimat')

question ="ma hi althaqafat almaghribiatu ?"
print ('\nQuestion 1:\n', question)
_, _ , _ , _, ans  = bert_question_answer( question, contexte)
print('\nAnswer from BERT: ', ans ,  '\n')


tul almamari: 17 kalimat

Question 1:
 ma hi althaqafat almaghribiatu ?

Answer from BERT:  mazij min alearab walbarbar al ' asliiyn wa ' afriqia janub alsahra ' i 



## Question answering en Arabe

In [12]:
print (f'طول الممر: {len(contexte.split())} كلمات')

question= "ما هي الثقافة المغربية؟"

contexte = """ 
الثقافة المغربية هي مزيج من العرب والبربر الأصليين وإفريقيا جنوب الصحراء والتأثيرات الأوروبية.""" 

print ('\nQuestion 1:\n', question)
_, _ , _ , _, ans  = bert_question_answer( question, contexte)
print('\nAnswer from BERT: ', ans ,  '\n')

طول الممر: 17 كلمات

Question 1:
 ما هي الثقافة المغربية؟

Answer from BERT:  هي مزيج من العرب والبربر الاصليين وافريقيا جنوب الصحراء والتاثيرات الاوروبية 




# Petit application de Question-Answering 


In [15]:
instructions =  widgets.HTML("<h1>Question-Answering app</h1>")
display(instructions)

instructions3 =  widgets.HTML("<h4>CONTEXTE : </h4>")
display(instructions3)

contexte = widgets.Textarea(
                            value=""" 
                            """,
                            placeholder='Paste your text here',
                            disablad=False)
display(contexte)

instructions2 =  widgets.HTML("<h4>QUESTION : </h4>")
display(instructions2)

question =  widgets.Text(value=" ",placeholder="Saissiez votre question",disablad=False)
display(question)

button = widgets.Button(description ='Get an answer', 
                        button_style='primary')
display(button)


def on_button_click(b):
    
    print('\nQUESTION :\n', question.value)
    _, _ , _ , _, ans  = bert_question_answer( question.value, contexte.value)
    
    print('\n ANSWER FROM BERT: ',ans , '\n')
    
button.on_click(on_button_click)

HTML(value='<h1>Question-Answering app</h1>')

HTML(value='<h4>CONTEXTE : </h4>')

Textarea(value=' \n                            ', placeholder='Paste your text here')

HTML(value='<h4>QUESTION : </h4>')

Text(value=' ', placeholder='Saissiez votre question')

Button(button_style='primary', description='Get an answer', style=ButtonStyle())


QUESTION :
  Qu'est-ce que Question Answer ?

 ANSWER FROM BERT:  une discipline informatique 

